## Project Name: House prices : Advanced regression techniques
          The main aim of this project is to predict the house price based on various features which we will discuss as we go ahead


        1.Data analysis
        2.Feature engenering
        3.Model Building
        4.Model deployment

# Data analysis phase ( understand more about the data)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
## display all the colomns of the data frame
pd.pandas.set_option('display.max_columns',None)

In [ ]:
dataset = pd.read_csv("train.csv")

NameError: name 'pd' is not defined

In [ ]:
print(dataset.shape)

In [ ]:
dataset.head ()


            1.Missing values
            2.All the numerical variables
            3.distribution of the numerical variables
            4.categorical values
            5.cardinality of categorcal values
            6.outliers
            7.the relationship between dependent and independent variables

In [ ]:
##Here we will check the percentage of nan values present in each feature
## 1 -step make the list of features which has missing values
features_with_na=[features for features in dataset.columns if dataset[features].isnull().sum()>1]
## 2- step print the feature name and the percentage of missing values
for feature in features_with_na:
    print (feature , np.round(dataset[feature].isnull().mean(),4),'% missing values')

## since there are many missing values we need to find out the relationship between the missing values and the sales price
           lets plot some diagram for this relationship

In [ ]:
for feature in features_with_na:
    data = dataset.copy()
## lets make a variable that indicates 1 if the observation was missing or zero otherwise
data[feature]=np.where (data[feature].isnull(),1,0)
# lets calculate the mean sales price where the information is missing or present
data.groupby(feature)['SalePrice'].median().plot.bar()
plt.title(feature)
plt.show()

## ere With the relation between the missing values and the dependent variable is clearly visible.So We need to replace these nan values with something meaningful which we will do in the Feature Engineering section

From the above dataset some of the features like Id is not required

In [ ]:
print ("Id of Houses{}".format(len(dataset.Id)))

## Numerical variables

In [ ]:
# list of numercal variables
numerical_features = [feature for feature in dataset.columns if dataset [feature ].dtypes != 'o']
print ('Number of numerical variables', len(numerical_features))

# visualize the numerical variables
dataset[numerical_features].head()

## Temporal variables
        From the Dataset we have 4 year variables. We have extracted information from the datetime variables like no of years or no of days. One example in this specific scenario can be difference in years between the year the house was built and the year the house was sold.

In [ ]:
# list of variables that contains datetime variables
year_feature = [feature for feature in numerical_features if 'Yr' in feature or 'Year' in feature]

year_feature

In [ ]:
# lets explore the content of these year variables

for feature in year_feature :
    print( feature, dataset[feature].unique())

In [ ]:
## Lets analyze the Temporal Datetime Variables
## We will check whether there is a relation between year the house is sold and the sales price

dataset.groupby('YrSold')['SalePrice'].median().plot()
plt.xlabel('Year Sold')
plt.ylabel('Median House Price')
plt.title('House Price vs Year Sold')

In [ ]:
# here we will compare the diff between the all years features with sales price

for feature in year_feature:
    if feature != 'YrSold':
        daat = dataset.copy()
    # we will compare the diff between the year variable and year the house was sold for
    data[feature] = data[feature]-data['YrSold']
    plt.scatter(data[feature ],data['SalePrice'])
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.show()

In [ ]:
# numerical variables are besically 2 types
# 1. Discrete variables and 2. continious variables

discrete_feature = [feature for feature in numerical_features if len(dataset[feature].unique())<25 and feature not in year_feature+['Id']]
print(" Discrete Varaibles Count: {}".format(len(discrete_feature)))


NameError: name 'numerical_features' is not defined

In [ ]:
discrete_feature

In [ ]:
dataset[discrete_feature].head()

In [ ]:
## lets find the relationship berween them and the sale price

for feature in discrete_feature:
    data=dataset.copy()
    data.groupby(feature)['SalePrice'].median().plot.bar()
    plt.xlabel(feature)
    plt.ylabel('sale price')
    plt.title(feature)
    plt.show()

## Continious Variable

In [ ]:
continuous_feature = [ feature for feature in numerical_features if feature not in discrete_feature+year_feature+['Id']]
print('Continuous feature Count {}'.format(len(continuous_feature)))

In [ ]:
## let's understand the continuous values by creating histograms to understand the distribution

for feature in continuous_feature:
    data = dataset.copy()
    data[feature].hist(bins=25)
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.title(feature)
    plt.show()

In [ ]:
## as from the above hist we can see the data is very skewed
## we will be using logarithmic trainsformation

for feature in continuous_feature:
    data=dataset.copy()
    if 0 in data[feature].unique():
        pass
    else:
        data[feature]=np.log(data[feature])
        data['SalePrice']=np.log1p(data['SalePrice'])
        plt.scatter(data[feature],data['SalePrice'])
        plt.xlabel(feature)
        plt.ylabel('SalesPrice')
        plt.title(feature)
        plt.show()


### Outliers


In [ ]:
for feature in continuous_feature:
    data = dataset.copy()
    if 0 in data[feature].unique():
        pass
    else:
        data[feature]= np.log(data[feature])
        data.boxplot(column=feature)
        plt.ylabel(feature)
        plt.title(feature)
        plt.show()

### Categorical variables

In [ ]:
categorical_features=[feature for feature in dataset.columns if data[feature].dtypes=='O']
categorical_features

In [ ]:
dataset[categorical_features].head()

In [ ]:
for feature in categorical_features:
    print ('The feature is {} and the number of categories are {}'.format(feature,len(dataset[feature].unique())))

In [ ]:
## Find out the relationship betweeen categorical variable and dependent feature like SalesPrice

for feature in categorical_features:
    data=dataset.copy()
    data.groupby(feature)['SalePrice'].median().plot.bar()
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.title(feature)
    plt.show()

## Advcanced House Prices : Feature engineering



          1.Missing values
          2.Temporal variables
          3.Categorical variables: remove rare labels
          4.Standarise the values of the variables to the same range


In [ ]:
y_test

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
## to visualize all the columns in the dataframe

pd.pandas.set_option('display.max_columns',None)

In [ ]:
dataset=pd.read_csv(r'/train.csv')
dataset.head()


In [ ]:
## there will always be some chance of data leakage so we will need to split the data first then apply feature engineering

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(dataset,dataset['SalePrice'],test_size=0.1,random_state=0)
y_test

In [ ]:
x_train.shape,x_test.shape

### Missing values

In [ ]:
## Let us capture all the nan values
## First lets handle Categorical features which are missing
features_nan=[feature for feature in dataset.columns if dataset[feature].isnull().sum()>1 and dataset[feature].dtypes=='O']

for feature in features_nan:
    print("{}: {}% missing values".format(feature,np.round(dataset[feature].isnull().mean(),4)))

In [ ]:
## Replace missing values with a new label

def replace_cat_feature(dataset,features_nan):
    data=dataset.copy()
    data[features_nan]=data[features_nan].fillna('Missing')
    return data

dataset=replace_cat_feature(dataset,features_nan)
data[features_nan].isnull().sum()

In [ ]:
dataset.head()

In [ ]:
## numerical features that has missing values

numerical_with_nan=[feature for feature in dataset.columns if dataset[feature].isnull().sum()>1 and dataset[feature].dtypes!='o']

## we will print the numerical nan vlaues and % of missing values

for feature in numerical_with_nan:
    print( " {}: {} %  missing value".format(feature,np.round(dataset[feature].isnull().mean(),4)))

In [ ]:
## replacing the numerical missing values
## replace them by usung median as there are outliers

for feature in numerical_with_nan:
    median_value=dataset[feature].median()
    ## create a new featrue to capture the nan values
    dataset[feature+'nan']=np.where(dataset[feature].isnull(),1,0)
    dataset[feature].fillna(median_value,inplace=True)






dataset[numerical_with_nan].isnull().sum()

In [ ]:
dataset.head(50)

In [ ]:
## Temporal variables ( Date time variables)
for feature in ['YearBuilt','YearRemodAdd','GarageYrBlt']:
    dataset[feature]=dataset['YrSold']-dataset[feature]



In [ ]:
dataset.head()

In [ ]:
dataset[['YearBuilt','YearRemodAdd','GarageYrBlt']].head()

### Numerical features
        since numeircal variables are skewed we will perform log normal distribution

In [ ]:
dataset.head()

In [ ]:
import numpy as np

num_features=['LotFrontage','LotArea','1stFlrSF','GrLivArea','SalePrice']

for feature in num_features:
    dataset[feature]=np.log(dataset[feature])

NameError: name 'dataset' is not defined

In [ ]:
dataset.head()

### Handling rare Categorical faetures
         we will categorical variables that are present less than 1% of the observations

In [ ]:
categorical_features=[feature for feature in dataset.columns if dataset[feature].dtype=='O']

In [ ]:
categorical_features

In [ ]:
for feature in categorical_features:
    temp=dataset.groupby(feature)['SalePrice'].count()/len(dataset)
    temp_df=temp[temp>0.1].index
    dataset[feature]=np.where(dataset[feature].isin(temp_df),dataset[feature],'Rare_var')

In [ ]:
dataset.head(100)

In [ ]:
for feature in categorical_features:
    labels_ordered=dataset.groupby([feature])['SalePrice'].mean().sort_values().index
    labels_ordered={k:i for i,k in enumerate(labels_ordered,0)}
    dataset[feature]=dataset[feature].map(labels_ordered)

In [ ]:
dataset.head(10)

In [ ]:
scaling_feature=[feature for feature in dataset.columns if feature not in ['Id','SalePrice']]
len(scaling_feature)

In [ ]:
scaling_feature

### Scaling Feature


In [ ]:
feature_scale=[feature for feature in dataset.columns if feature not in ['Id','SalePrice']]



In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
scaler.fit(dataset[feature_scale])


In [ ]:
scaler.transform(dataset[feature_scale])

In [ ]:
data=pd.concat([dataset[['Id','SalePrice']].reset_index(drop=True),
                  pd.DataFrame(scaler.transform(dataset[feature_scale]),columns=feature_scale)],axis=1)

In [ ]:
data.head()

In [ ]:
data.to_csv('x_train.csv',index=False)

## Feature Selection

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## for feature selection

from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

## to visualize all columns in the dataframe

pd.pandas.set_option('display.max_columns',None)

In [ ]:
dataset=pd.read_csv('x_train.csv')

In [ ]:
dataset.head()

In [ ]:
# capture the dependent feature
y_train=dataset[['SalePrice']]

In [ ]:
## drop the dependent feature form the dataset

x_train=dataset.drop(['Id','SalePrice'],axis=1)

In [ ]:
print (x_train.isnull().sum())

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
Imputer = SimpleImputer(strategy='mean')  # Replace 'mean' with 'median' or 'most_frequent' if needed
x_train=Imputer.fit_transform(x_train) # Apply imputation

In [ ]:
###  Feature Selection
# first, I specify the Lasso Regression model, and I
# select a suitable alpha (equivalent of penalty).
# The bigger the alpha the less features that will be selected.

# Then I use the selectFromModel object from sklearn, which
# will select the features which coefficients are non-zero

feature_sel_model = SelectFromModel(Lasso(alpha=0.005,random_state=0)) ## to set the seed , the random state in tis function

feature_sel_model.fit(x_train,y_train)


In [ ]:
feature_sel_model.get_support()

In [ ]:
x_train=pd.DataFrame(x_train, columns)

In [ ]:
# lets print the total and no. of selected features
# make a list of selected features

selected_feat= x_train.columns[(feature_sel_model.get_support())]

# let's print some stats

print('total features : {}'. format((x_train.shape[1])))
print('selected features : {}'.format(len(selected_feat)))
print("features with coefficients shrank to zero : {}".format (np.sum(sel_.estimator_.coef_== 0)))

In [ ]:
# let's print the number of total and selected features

# this is how we can make a list of the selected features
selected_feat = x_train.columns[(feature_sel_model.get_support())]

# let's print some stats
print('total features: {}'.format((x_train.shape[1])))
print('selected features: {}'.format(len(selected_feat)))
print('features with coefficients shrank to zero: {}'.format(
    np.sum(sel_.estimator_.coef_ == 0)))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test(X,y,test_size=0.2,random_state=0)
RF=RandomForestClassifier()
model.fit(x_train,y_train)

